In [18]:
# Step 1: Install required libraries
!pip install transformers rouge-score nltk bert-score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [20]:
# Import necessary libraries
import json
from transformers import pipeline
import time

In [21]:
# Load the dataset
input_file_path = '/kaggle/input/nuclearkinet/cleaned_json.json'

In [22]:
with open(input_file_path, "r") as f:
    data = json.load(f)

content_key = "Content"

# Load pre-trained models for question generation and question answering
question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qg-hl")
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

output_data = []

try:
    for i, entry in enumerate(data["Sheet1"]):
        print(f"Processing entry {i+1}/{len(data['Sheet1'])}")
        
        if content_key not in entry:
            print(f"Key '{content_key}' not found in entry: {entry}")
            continue

        context = entry[content_key]
        
        if not context or len(context.strip()) == 0:
            print(f"Skipping empty or invalid content in entry: {entry}")
            continue

        print(f"Processing context: {context[:100]}...")

        try:
            # Generate one question
            question = question_generator(f"generate question: {context}", max_length=100)[0]['generated_text']
            print(f"Generated question: {question}")

            if not question.endswith('?'):
                question += "?"

            # Generate answer
            answer = qa_pipeline(question=question, context=context, max_answer_len=200)['answer']
            print(f"Generated answer: {answer}")

            output_data.append({
                "prompt": question,
                "response": answer,
            })

        except Exception as e:
            print(f"Error generating question or answer: {e}")
            continue

        print(f"Generated {len(output_data)} question-answer pairs so far")

        time.sleep(1)  # Add a 1-second delay between entries

except Exception as e:
    print(f"An error occurred during processing: {e}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Processing entry 1/1220
Processing context: March 2021 REGDOC-1.4.1, Licence Application Guide: Class II Nuclear Facilities and Prescribed Equip...
Generated question: What is the name of the document that outlines requirements for applying for a Class II facility and/or prescribed equipment licence?
Generated answer: REGDOC-1.4.1
Generated 1 question-answer pairs so far
Processing entry 2/1220
Processing context: March 2021 REGDOC-1.4.1, Licence Application Guide: Class II Nuclear Facilities and Prescribed Equip...
Generated question: What is the name of the document that outlines requirements for applying for a Class II facility and/or prescribed equipment licence?
Generated answer: REGDOC-1.4.1
Generated 2 question-answer pairs so far
Processing entry 3/1220
Processing context: March 2021 REGDOC-1.4.1, Licence Application Guide: Class II Nuclear Facilities and Prescribed Equip...
Generated question: What is the name of the document that outlines requirements for applying for a Class

In [24]:
# Save the output data to a JSON file
output_file_path = '/kaggle/working/output.json'
with open(output_file_path, "w") as f:
    json.dump(output_data, f)

In [25]:
print(f"Output saved to {output_file_path}")
print(f"Total question-answer pairs generated: {len(output_data)}")

Output saved to /kaggle/working/output.json
Total question-answer pairs generated: 1220
